In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

### 把不要的欄位刪掉，改欄位名稱，改column type，把沒有label的資料刪掉

In [5]:
laptop_test = pd.read_csv("dataset/laptop_test.csv", encoding="utf-8")
#刪掉不要的欄位
laptop_test = laptop_test.drop(columns=['aspectTerms/aspectTerm/0/_from','aspectTerms/aspectTerm/0/_to',
                                          'aspectTerms/aspectTerm/1/_from','aspectTerms/aspectTerm/1/_to',
                                          'aspectTerms/aspectTerm/2/_from','aspectTerms/aspectTerm/2/_to',
                                          'aspectTerms/aspectTerm/3/_from','aspectTerms/aspectTerm/3/_to',
                                          'aspectTerms/aspectTerm/4/_from','aspectTerms/aspectTerm/4/_to',
                                          'aspectTerms/aspectTerm/5/_from','aspectTerms/aspectTerm/5/_to','_id'])
#把欄位重新命名
laptop_test = laptop_test.rename(columns={'aspectTerms/aspectTerm/0/_term': 'aspect_0', 'aspectTerms/aspectTerm/0/_polarity': 'polarity_0',
                                            'aspectTerms/aspectTerm/1/_term': 'aspect_1', 'aspectTerms/aspectTerm/1/_polarity': 'polarity_1',
                                            'aspectTerms/aspectTerm/2/_term': 'aspect_2', 'aspectTerms/aspectTerm/2/_polarity': 'polarity_2',
                                            'aspectTerms/aspectTerm/3/_term': 'aspect_3', 'aspectTerms/aspectTerm/3/_polarity': 'polarity_3',
                                            'aspectTerms/aspectTerm/4/_term': 'aspect_4', 'aspectTerms/aspectTerm/4/_polarity': 'polarity_4',
                                            'aspectTerms/aspectTerm/5/_term': 'aspect_5', 'aspectTerms/aspectTerm/5/_polarity': 'polarity_5'})
# 把欄位型態改成str
laptop_test['aspect_0'] = laptop_test['aspect_0'].astype(str)
laptop_test['aspect_1'] = laptop_test['aspect_1'].astype(str)
laptop_test['aspect_2'] = laptop_test['aspect_2'].astype(str)
laptop_test['aspect_3'] = laptop_test['aspect_3'].astype(str)
laptop_test['aspect_4'] = laptop_test['aspect_4'].astype(str)
laptop_test['aspect_5'] = laptop_test['aspect_5'].astype(str)

laptop_test['polarity_0'] = laptop_test['polarity_0'].astype(str)
laptop_test['polarity_1'] = laptop_test['polarity_1'].astype(str)
laptop_test['polarity_2'] = laptop_test['polarity_2'].astype(str)
laptop_test['polarity_3'] = laptop_test['polarity_3'].astype(str)
laptop_test['polarity_4'] = laptop_test['polarity_4'].astype(str)
laptop_test['polarity_5'] = laptop_test['polarity_5'].astype(str)

laptop_test=laptop_test.drop(laptop_test[laptop_test['aspect_0']=='nan'].index) #把沒有Label的資料刪掉
laptop_test = laptop_test.reset_index(drop=True)
print('有label的筆數:',len(laptop_test))
laptop_test.head()

有label的筆數: 422


,text,aspect_0,polarity_0,aspect_1,polarity_1,aspect_2,polarity_2,aspect_3,polarity_3,aspect_4,polarity_4,aspect_5,polarity_5
0,"Boot time is super fast, around anywhere from ...",Boot time,positive,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,tech support would not fix the problem unless ...,tech support,negative,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,Set up was easy.,Set up,positive,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,Did not enjoy the new Windows 8 and touchscree...,Windows 8,negative,touchscreen functions,negative,nan,nan,nan,nan,nan,nan,nan,nan
4,Other than not being a fan of click pads (indu...,internal speakers,negative,price tag,positive,click pads,negative,nan,nan,nan,nan,nan,nan


### 看每筆資料的apsect數各有多少筆資料

In [9]:
laptop_copy = laptop_test

aspect_6 = laptop_copy[laptop_copy['aspect_5'] != 'nan']
laptop_copy = laptop_copy.drop(laptop_copy[laptop_copy['aspect_5'] != 'nan'].index)

aspect_5 = laptop_copy[laptop_copy['aspect_4'] != 'nan']
laptop_copy = laptop_copy.drop(laptop_copy[laptop_copy['aspect_4'] != 'nan'].index)

aspect_4 = laptop_copy[laptop_copy['aspect_3'] != 'nan']
laptop_copy = laptop_copy.drop(laptop_copy[laptop_copy['aspect_3'] != 'nan'].index)

aspect_3 = laptop_copy[laptop_copy['aspect_2'] != 'nan']
laptop_copy = laptop_copy.drop(laptop_copy[laptop_copy['aspect_2'] != 'nan'].index)

aspect_2 = laptop_copy[laptop_copy['aspect_1'] != 'nan']
laptop_copy = laptop_copy.drop(laptop_copy[laptop_copy['aspect_1'] != 'nan'].index)

aspect_1 = laptop_copy

print('所有資料數:',len(laptop_test))
print('1個aspect的資料數:',len(aspect_1))
print('2個aspect的資料數:',len(aspect_2))
print('3個aspect的資料數:',len(aspect_3))
print('4個aspect的資料數:',len(aspect_4))
print('5個aspect的資料數:',len(aspect_5))
print('6個aspect的資料數:',len(aspect_6))

所有資料數: 422
1個aspect的資料數: 266
2個aspect的資料數: 105
3個aspect的資料數: 34
4個aspect的資料數: 10
5個aspect的資料數: 6
6個aspect的資料數: 1


### 把每筆資料對每個aspect切開，讓一筆資料只能有一個aspect，如果一筆資料有多個aspect，重複text形成一對多的格式

In [11]:
# 創一個空的dataframe
laptop_test_1 = pd.DataFrame(columns=['text','aspect','polarity'])

# 把每筆資料的text 對 aspect terms 分開成 一個text對一個aspect、polarity
for i in tqdm(range(len(laptop_test))):
    count = 0
    target_a = laptop_test.loc[i,'aspect_0'] # aspect_0值
    target_p = laptop_test.loc[i,'polarity_0'] # polarity_0的值
    while((count<=5) and (target_a != 'nan')): #如果count<=12 (因為dataframe只有0-12個aspect) 且 目標aspect值不是'nan'
        pd_temp = pd.DataFrame([[laptop_test.loc[i,'text'], target_a, target_p]],
                              columns=['text','aspect','polarity']) # 將text、aspect、polarity變成dataframe格式
        laptop_test_1 = laptop_test_1.append(pd_temp, ignore_index=True) # 將dataframe加在下面
        count += 1
        if count>5: # 判斷aspect數是否有超過資料集裡的aspect數，如果超過就break
            break
        a = 'aspect_'+ str(count) #目標放在下一個aspect column name
        p = 'polarity_'+str(count) # 下一個 polarity column name
        target_a = laptop_test.loc[i,a] # 下一個aspect 值
        target_p = laptop_test.loc[i,p] # 下一個polarity 值
    laptop_test.loc[i,'aspect count'] = str(count)
laptop_test_1

100%|██████████| 422/422 [00:00<00:00, 668.97it/s]


,text,aspect,polarity
0,"Boot time is super fast, around anywhere from ...",Boot time,positive
1,tech support would not fix the problem unless ...,tech support,negative
2,Set up was easy.,Set up,positive
3,Did not enjoy the new Windows 8 and touchscree...,Windows 8,negative
4,Did not enjoy the new Windows 8 and touchscree...,touchscreen functions,negative
...,...,...,...
649,I've had it for about 2 months now and found n...,software,neutral
650,I've had it for about 2 months now and found n...,updates,neutral
651,the latest version does not have a disc drive.,disc drive,neutral
652,Screen - although some people might complain a...,Screen,positive


In [12]:
# 看分開aspect term資料筆數有沒有一樣多
print('切成單一資料筆數:',len(laptop_test_1))
print('每個aspect相加:',len(aspect_1)+len(aspect_2)*2+len(aspect_3)*3+len(aspect_4)*4+len(aspect_5)*5+
      len(aspect_6)*6)

切成單一資料筆數: 654
每個aspect相加: 654


In [13]:
# 把polarity是conflict的資料刪除，(因為conflict資料量太少)
laptop_test_1=laptop_test_1.drop(laptop_test_1[laptop_test_1['polarity']=='conflict'].index)
print('扣掉label為conflict的資料數:',len(laptop_test_1))
laptop_test_1

扣掉label為conflict的資料數: 638


,text,aspect,polarity
0,"Boot time is super fast, around anywhere from ...",Boot time,positive
1,tech support would not fix the problem unless ...,tech support,negative
2,Set up was easy.,Set up,positive
3,Did not enjoy the new Windows 8 and touchscree...,Windows 8,negative
4,Did not enjoy the new Windows 8 and touchscree...,touchscreen functions,negative
...,...,...,...
649,I've had it for about 2 months now and found n...,software,neutral
650,I've had it for about 2 months now and found n...,updates,neutral
651,the latest version does not have a disc drive.,disc drive,neutral
652,Screen - although some people might complain a...,Screen,positive


In [15]:
# 存成csv
laptop_test_1 = laptop_test_1.reset_index(drop=True)
laptop_test_1.to_csv('laptop_test_processed.csv',index=0)